In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import json
import time
import os
import re

In [3]:
path = 'D:/KMUTT/Year4 Term2/CPE402/AI Chatbot/Web-Scraping/chromedriver-win64/'
os.chdir(path)

In [4]:
def fetch_links(url, province_text):
    # Open webdriver
    driver = webdriver.Chrome(path + "chromedriver.exe")
    driver.get(url)
    driver.implicitly_wait(5)

    actions = ActionChains(driver)
    
    # Select province
    province_dropdown = driver.find_elements(By.ID, 'dropdownMenuButton')[1]
    actions.move_to_element(province_dropdown).click().perform()
    province_element = driver.find_element(By.XPATH, f'//div[@class="dropdown-menu show"]//*[contains(text(), "{province_text}")]')
    actions.move_to_element(province_element).click().perform()

    page_links = []
    current_page = 1
    while True:
        try:
            # Find links on the page
            links = driver.find_elements(By.XPATH, '//div[@class="single-destinations-list style-two animated slideInUp border-solid"]/a')

            # Check for duplicates and add unique links to the list
            page_links.extend(link.get_attribute('href') for link in links if link.get_attribute('href') not in page_links)
            
            # Click on the next page link
            next_page_link = driver.find_element(By.XPATH, f'//a[@hreflang="th"]/span[text()="{current_page+1}"]')
            actions.move_to_element(next_page_link).click().perform()
            current_page += 1
            # Add a short delay to allow the page to load
            time.sleep(2)
        except NoSuchElementException:
            break

    # Quit webdriver        
    driver.quit()

    # page_links = list(page_links_set)
    return page_links

In [5]:
url_all = "https://thailandtourismdirectory.go.th/th/attraction?page=1&province_id=50"

result_links = fetch_links(url_all, 'เชียงใหม่')

C:\Users\MSII\AppData\Local\Temp\ipykernel_16188\2257744001.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path + "chromedriver.exe")


In [6]:
print(result_links)
print(len(result_links))

['https://thailandtourismdirectory.go.th/th/attraction/5171', 'https://thailandtourismdirectory.go.th/th/attraction/98846', 'https://thailandtourismdirectory.go.th/th/attraction/4569', 'https://thailandtourismdirectory.go.th/th/attraction/140', 'https://thailandtourismdirectory.go.th/th/attraction/2702', 'https://thailandtourismdirectory.go.th/th/attraction/5175', 'https://thailandtourismdirectory.go.th/th/attraction/98908', 'https://thailandtourismdirectory.go.th/th/attraction/98843', 'https://thailandtourismdirectory.go.th/th/attraction/98845', 'https://thailandtourismdirectory.go.th/th/attraction/211', 'https://thailandtourismdirectory.go.th/th/attraction/87046', 'https://thailandtourismdirectory.go.th/th/attraction/98021', 'https://thailandtourismdirectory.go.th/th/attraction/98895', 'https://thailandtourismdirectory.go.th/th/attraction/98900', 'https://thailandtourismdirectory.go.th/th/attraction/98838', 'https://thailandtourismdirectory.go.th/th/attraction/98017', 'https://thaila

In [9]:
def extract_data(url):
    # Open webdriver
    driver = webdriver.Chrome(path + "chromedriver.exe")
    driver.get(url)
    driver.implicitly_wait(5)

    ### ชื่อสถานที่
    name_place = driver.find_element(By.CSS_SELECTOR, 'p.large-text-36.cl-black.dp').text
    ### ที่อยู่
    addresses = driver.find_elements(By.CSS_SELECTOR, 'div.pd-r-15')
    address_text = ' '.join([address.text for address in addresses])
    ### sub ของ day_text และ contract_text
    subtitle = driver.find_elements(By.CSS_SELECTOR, 'h6.card-subtitle.topic-text.pd-bt-15')
    sub_day_time = subtitle[0].text
    sub_contract = subtitle[1].text
    ### วันเวลาทำการ
    days = driver.find_elements(By.CSS_SELECTOR, 'div.col-xl-4.col-lg-5.col-md-5.col-sm-5.col-5')
    day_text = [day.text for day in days]
    times = driver.find_elements(By.CSS_SELECTOR, 'div.col-xl-8.col-lg-7.col-md-7.col-sm-7.col-7')
    time_text = [re.sub(r'\n', ' ', time.text) for time in times]
    day_time = ', '.join([f"{day} {time}" for day, time in zip(day_text, time_text)])
    ### หมายเหตุวันเวลาทำการ
    note_time = driver.find_element(By.XPATH, '//div[@class="card-body"]/div[@class="row"]/p[@class="normal-text"]').text
    ### ข้้อมูลติดต่อ
    contracts = driver.find_elements(By.XPATH, '//div[@class="card-body"]/div/p[@class="normal-text pd-bt-15"]')
    contract_text = ', '.join([contract.text for contract in contracts])
    ### Lable ของข้อมูลสำคัญเช่น ข้อมูลสถานที่, ประเภทแหล่งท่องเที่ยว, ลักษณะเด่น, ประวัติ
    all_lable = driver.find_elements(By.CSS_SELECTOR, 'h5.topic-text.pd-bt-15')
    lable_text = [lable.text for lable in all_lable]
    ### ข้อมูลสถานที่, ประเภทแหล่งท่องเที่ยว, ลักษณะเด่น, ประวัติ
    all_info = driver.find_elements(By.XPATH, '//div[@class="col-md-12 col-sm-12 col-12"]/div/p[@class="normal-text pd-bt-15"]')
    info_text = [re.sub(r'[\n-]', ' ', info.text) for info in all_info]
    ### ข้อมูลเพิ่มเติม
    more_data = driver.find_elements(By.CSS_SELECTOR, 'p.normal-text.cl-black')
    more_text = ' '.join([more.text for more in more_data])
    ### ข้อมูลแนะนำ
    recom_data = driver.find_elements(By.CSS_SELECTOR, 'div.col-lg-6.pd-bt-15')
    list_recom = [f"{recom_data[index].find_elements(By.CSS_SELECTOR, 'div.row')[0].text} จะมี{' '.join([recom.text for recom in recom_data[index].find_elements(By.CSS_SELECTOR, 'div.col-lg-12')])}" for index in range(len(recom_data))]
    all_recom = ', '.join(list_recom)
    ### สิ่งอำนวยความสะดวก
    facility = driver.find_elements(By.XPATH, '//div[@class="col-lg-4"]/div[@class="row"]/p[@class="normal-text"]')
    facil_text = ', '.join([facil.text for facil in facility])
    ### สาธารณูปโภค
    utility = driver.find_elements(By.XPATH, '//div[@class="row"]/div[@class="col-lg-4 pd-bt-15"]')
    list_util = [f"{utility[index].find_elements(By.CSS_SELECTOR, 'div.row')[0].text} จะมี{' '.join([util.text for util in utility[index].find_elements(By.CSS_SELECTOR, 'div.col-lg-12')])}" for index in range(len(utility))]
    all_util = ', '.join(list_util)
    ### กิจกรรมท่องเที่ยว
    activity = driver.find_elements(By.XPATH, '//div[@class="row"]/div[@class="col-lg-6"]')
    list_act = [f"{activity[index].find_elements(By.CSS_SELECTOR, 'div.row')[0].text} จะมี{' '.join([act.text for act in activity[index].find_elements(By.CSS_SELECTOR, 'div.col-lg-12')])}" for index in range(len(activity))]
    all_act = ', '.join(list_act)

    # Check for null values and update
    # Check if contains any special characters 
    value_day_time = f"{sub_day_time}: {day_time} {note_time}" if day_time and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', day_time)) else ""
    value_contract = f"{sub_contract}: {contract_text}" if contract_text and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', contract_text)) else ""
    value_more_text = f"ข้อมูลเพิ่มเติม: {more_text}" if more_text and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', more_text)) else ""
    value_all_recom = f"ข้อมูลแนะนำ: {all_recom}" if all_recom and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', all_recom)) else ""
    value_facil_text = f"สิ่งอำนวยความสะดวก: {facil_text}" if facil_text and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', facil_text)) else ""
    value_all_util = f"สาธารณูปโภค: {all_util}" if all_util and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', all_util)) else ""
    value_all_act = f"กิจกรรมท่องเที่ยว: {all_act}" if all_act and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', all_act)) else ""

    # Store data in dictionary
    data_dict = {
        "place_name": name_place,
        "address": address_text,
        "all_day_time": value_day_time,
        "all_contract": value_contract,
        **{f"{lable_text[index]}": f"{lable_text[index]}: {info_text[index]}" if info_text[index] and not bool(re.search(r'^[^A-Za-z0-9ก-๙]+$', info_text[index])) else "" for index in range(len(info_text))}
    }

    data_dict['place_data'] = data_dict.pop('ข้อมูลสถานที่', "")
    data_dict['place_type'] = data_dict.pop('ประเภทแหล่งท่องเที่ยว', "")
    data_dict['feature'] = data_dict.pop('ลักษณะเด่น', "")
    data_dict['history'] = data_dict.pop('ประวัติ', "")

    other_data_dict = {
        "more_data": value_more_text,
        "recom_info": value_all_recom,
        "facility": value_facil_text,
        "utility": value_all_util,
        "activity": value_all_act
    }
    data_dict.update(other_data_dict)

    # Remove trailing ', ' from values in data_dict
    data_dict = {key: value.rstrip(', ') if isinstance(value, str) else value for key, value in data_dict.items()}

    # Close the webdriver
    driver.quit()

    return data_dict

In [10]:
# Iterate through the URLs and extract data
content_list = []
for url in result_links:
    data_dict = extract_data(url)
    content_list.append(data_dict)

C:\Users\MSII\AppData\Local\Temp\ipykernel_16188\858955454.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path + "chromedriver.exe")


In [11]:
# Save to JSON
json_table = json.dumps(content_list, ensure_ascii=False, indent=4)

with open('web_scraping.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_table)
print(json_table)

[
    {
        "place_name": "ศูนย์ศิลปะการแสดงกาดสวนแก้ว",
        "address": "ตำบล: สุเทพ อำเภอ: เมืองเชียงใหม่ จังหวัด: เชียงใหม่ 50200",
        "all_day_time": "วันเวลาทำการ: วันจันทร์ 10:00 น. - 23:00 น., วันอังคาร 10:00 น. - 23:00 น., วันพุธ 10:00 น. - 23:00 น., วันพฤหัสบดี 10:00 น. - 23:00 น., วันศุกร์ 10:00 น. - 23:00 น., วันเสาร์ 10:00 น. - 23:00 น., วันอาทิตย์ 10:00 น. - 23:00 น. หมายเหตุเวลาทำการ: -",
        "all_contract": "ข้อมูลติดต่อ: ตำบล/แขวงสุเทพ อำเภอ/เขตเมืองเชียงใหม่",
        "place_data": "ข้อมูลสถานที่: แหล่งเรียนรู้ทางด้านศิลปะการแสดงที่เปิดมายาวนานถึง 10 ปี และยังคงการเรียนการสอนทางด้านการแสดงครบถ้วน สมบูรณ์แบบมาจนทุกวันนี้ ทั้งการออกแบบด้านเทคนิค แสง เสียง ฉาก รวมทั้งกระบวนการเบื้องหลัง และการทำละครเวที โดยโรงเรียนแห่งนี้มีการปรับปรุงหลักสูตรมาอย่างต่อเนื่องเพื่อให้สอดคล้องกับยุคสมัย  นอกจากนี้ ภายในศูนย์ยังมีห้องสมุด หอศิลป์ วงดนตรี คณะละคร และโรงละครกาดสวนแก้ว ซึ่งเป็นโรงละครที่ได้รับการออกแบบและใช้อุปกรณ์ทางเทคนิคที่ทันสมัย ตลอดทั้งปีจะมีการแสดงทั้งใน แ

In [12]:
# Save to Excel (xlsx)
excel_table = pd.DataFrame(content_list)

excel_table.to_excel('web_scraping.xlsx', index=False)
print(excel_table)

                                  place_name  \
0                ศูนย์ศิลปะการแสดงกาดสวนแก้ว   
1    ศูนย์วิจัยข้าวสะเมิง (ทุ่งข้าวบาร์เลย์)   
2                               น้ำตกมณฑาธาร   
3        ถ้ำหลวงแม่ตูม (อุทยานแห่งชาติแม่โถ)   
4                          วัดพระธาตุดอยน้อย   
..                                       ...   
236                               วัดโลกโมฬี   
237   สวนพฤกษศาสตร์สมเด็จพระนางเจ้าสิริกิติ์   
238                               วัดทรายมูล   
239                        น้ำพุร้อนสันกำแพง   
240    โครงการหลวงห้วยผักไผ่ (สวนกุหลาบหลวง)   

                                               address  \
0    ตำบล: สุเทพ อำเภอ: เมืองเชียงใหม่ จังหวัด: เชี...   
1     ตำบล: สะเมิงใต้ อำเภอ: สะเมิง จังหวัด: เชียงใหม่   
2    ตำบล: สุเทพ อำเภอ: เมืองเชียงใหม่ จังหวัด: เชี...   
3      ตำบล: บ้านทับ อำเภอ: แม่แจ่ม จังหวัด: เชียงใหม่   
4    ตำบล: ดอยหล่อ อำเภอ: ดอยหล่อ จังหวัด: เชียงใหม...   
..                                                 ...   
236  ตำบล: ศรีภูม